In [1]:
import re

In [2]:
import os
from pathlib import Path

# Move up if we are inside the notebooks folder
if Path(os.getcwd()).name == "notebooks":
    os.chdir("..")

print(f"🏠 Root established at: {os.getcwd()}")

🏠 Root established at: /Users/ganeshprasadbhandari/Documents/D_drive/clark/careeragent-ai


In [3]:
# -----------------------------
# 0) Ensure directories exist
# -----------------------------
(Path("src/careeragent") / "__init__.py").parent.mkdir(parents=True, exist_ok=True)
Path("src/careeragent/__init__.py").write_text("", encoding="utf-8")

Path("src/careeragent/agents").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/agents/__init__.py").write_text("", encoding="utf-8")

Path("src/careeragent/services").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/services/__init__.py").write_text("", encoding="utf-8")

Path("src/careeragent/api").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/api/__init__.py").write_text("", encoding="utf-8")

Path("app").mkdir(parents=True, exist_ok=True)
Path("src/careeragent/artifacts").mkdir(parents=True, exist_ok=True)
(Path("src/careeragent/artifacts") / "runs").mkdir(parents=True, exist_ok=True)
(Path("src/careeragent/artifacts") / "reports").mkdir(parents=True, exist_ok=True)
(Path("src/careeragent/artifacts") / "exports").mkdir(parents=True, exist_ok=True)

# -----------------------------
# 1) src/careeragent/config.py
# -----------------------------
config_py = r'''
from __future__ import annotations

from functools import lru_cache
from pathlib import Path
from typing import Optional

from pydantic_settings import BaseSettings, SettingsConfigDict


def repo_root() -> Path:
    """
    Description: Resolve repository root from within src/ package.
    Layer: L0
    Input: None
    Output: Absolute Path to repo root
    """
    # src/careeragent/config.py -> src/careeragent -> src -> repo root
    return Path(__file__).resolve().parents[2]


def artifacts_root() -> Path:
    """
    Description: Canonical artifacts root required by CareerAgent-AI (local-first).
    Layer: L0
    Input: None
    Output: Path to src/careeragent/artifacts
    """
    root = Path(__file__).resolve().parents[0] / "artifacts"
    root.mkdir(parents=True, exist_ok=True)
    return root


class Settings(BaseSettings):
    """
    Description: Central configuration loader for CareerAgent-AI.
    Layer: L0
    Input: .env in repo root + environment variables
    Output: Strongly typed settings object
    """

    model_config = SettingsConfigDict(
        env_file=str(repo_root() / ".env"),
        env_file_encoding="utf-8",
        extra="ignore",
    )

    # Tracing
    langsmith_api_key: Optional[str] = None
    langsmith_project: str = "careeragent-ai"

    # LLM backends (local-first)
    ollama_base_url: Optional[str] = None

    # Search
    serper_api_key: Optional[str] = None

    # Notifications
    twilio_account_sid: Optional[str] = None
    twilio_auth_token: Optional[str] = None
    twilio_phone: Optional[str] = None  # From number
    user_phone: Optional[str] = None    # To number for local testing

    # Runtime
    environment: str = "local"


@lru_cache(maxsize=1)
def get_settings() -> Settings:
    """
    Description: Cached settings accessor for FastAPI/Streamlit.
    Layer: L0
    Input: None
    Output: Settings
    """
    return Settings()
'''
Path("src/careeragent/config.py").write_text(config_py.strip() + "\n", encoding="utf-8")

# -----------------------------
# 2) Refactor NotificationService (Twilio SDK preferred; safe fallback)
#    src/careeragent/services/notification_service.py
# -----------------------------
notification_py = r'''
from __future__ import annotations

from typing import Any, Dict, Literal, Optional

from careeragent.config import get_settings
from careeragent.orchestration.state import OrchestrationState


class NotificationService:
    """
    Description: L7 notifications service using Twilio SMS for critical run status changes.
    Layer: L7
    Input: OrchestrationState transitions and quota/security events
    Output: SMS send attempts logged to state.meta['notifications']
    """

    def __init__(self, *, dry_run: bool = False) -> None:
        """
        Description: Initialize notification service.
        Layer: L0
        Input: dry_run flag
        Output: NotificationService
        """
        self._settings = get_settings()
        self._dry_run = bool(dry_run)

    def _twilio_ready(self) -> bool:
        """
        Description: Check Twilio credential presence.
        Layer: L0
        Input: Settings
        Output: bool
        """
        s = self._settings
        return bool(s.twilio_account_sid and s.twilio_auth_token and s.twilio_phone)

    def send_sms(self, *, to_phone: str, body: str) -> Dict[str, Any]:
        """
        Description: Send SMS via Twilio (SDK if installed, otherwise safe error).
        Layer: L7
        Input: to_phone + body
        Output: dict result
        """
        payload = {"to": to_phone, "from": self._settings.twilio_phone, "body": body}

        if self._dry_run or not self._twilio_ready():
            return {"sent": False, "dry_run": True, "reason": "dry_run_or_missing_twilio_config", "payload": payload}

        try:
            from twilio.rest import Client  # type: ignore
        except Exception as e:
            return {"sent": False, "dry_run": True, "reason": f"twilio_sdk_missing:{e}", "payload": payload}

        client = Client(self._settings.twilio_account_sid, self._settings.twilio_auth_token)
        msg = client.messages.create(to=to_phone, from_=self._settings.twilio_phone, body=body)
        return {"sent": True, "sid": getattr(msg, "sid", None), "payload": payload}

    def notify(
        self,
        *,
        state: OrchestrationState,
        event: Literal["needs_human_approval", "completed", "quota_error"],
        extra: Optional[Dict[str, Any]] = None,
        to_phone: Optional[str] = None,
    ) -> Dict[str, Any]:
        """
        Description: Send critical status notification and log it in OrchestrationState.meta.
        Layer: L7
        Input: state + event + extra + optional to_phone override
        Output: result dict
        """
        extra = extra or {}
        to = (to_phone or self._settings.user_phone or "").strip()
        if not to:
            result = {"sent": False, "dry_run": True, "reason": "missing_user_phone", "event": event}
        else:
            if event == "needs_human_approval":
                body = f"CareerAgent-AI: Run {state.run_id} needs human approval. Open the dashboard to review."
            elif event == "completed":
                body = f"CareerAgent-AI: Run {state.run_id} completed successfully."
            else:
                provider = extra.get("provider", "unknown")
                body = f"CareerAgent-AI: Run {state.run_id} blocked due to API quota error ({provider})."
            result = self.send_sms(to_phone=to, body=body)

        state.meta.setdefault("notifications", [])
        state.meta["notifications"].append({"event": event, "to": to, "result": result, "extra": extra})
        state.touch()
        return result
'''
Path("src/careeragent/services/notification_service.py").write_text(notification_py.strip() + "\n", encoding="utf-8")

# -----------------------------
# 3) FastAPI Brain: src/careeragent/api/main.py
# -----------------------------
api_main_py = r'''
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Optional

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field

from careeragent.config import artifacts_root, get_settings
from careeragent.orchestration.state import OrchestrationState
from careeragent.agents.security_agent import SanitizeAgent
from careeragent.services.notification_service import NotificationService

from careeragent.agents.parser_agent_service import ParserAgentService
from careeragent.agents.parser_evaluator_service import ParserEvaluatorService

from careeragent.agents.matcher_agent_schema import JobDescription
from careeragent.agents.matcher_agent_service import MatcherAgentService
from careeragent.agents.strategy_agent_service import StrategyAgentService
from careeragent.agents.cover_letter_service import CoverLetterService
from careeragent.agents.apply_executor_service import ApplyExecutorService

from careeragent.services.xai_service import XAIService
from careeragent.services.exporter import CareerDossierExporter


app = FastAPI(title="CareerAgent-AI Beta Brain", version="0.1.0")


def _run_dir(run_id: str) -> Path:
    """
    Description: Resolve run directory under canonical artifacts path.
    Layer: L0
    Input: run_id
    Output: Path artifacts/runs/<run_id>
    """
    p = artifacts_root() / "runs" / run_id
    p.mkdir(parents=True, exist_ok=True)
    return p


def _state_path(run_id: str) -> Path:
    """
    Description: Resolve persistent state.json for a run.
    Layer: L0
    Input: run_id
    Output: Path artifacts/runs/<run_id>/state.json
    """
    return _run_dir(run_id) / "state.json"


def persist_state(state: OrchestrationState) -> None:
    """
    Description: Persist orchestration state to disk for polling from UI.
    Layer: L8
    Input: OrchestrationState
    Output: state.json written
    """
    _state_path(state.run_id).write_text(json.dumps(state.model_dump(), indent=2), encoding="utf-8")


def load_state(run_id: str) -> Dict[str, Any]:
    """
    Description: Load state.json for status polling.
    Layer: L8
    Input: run_id
    Output: dict
    """
    p = _state_path(run_id)
    if not p.exists():
        raise FileNotFoundError(run_id)
    return json.loads(p.read_text(encoding="utf-8"))


class AnalyzeRequest(BaseModel):
    """
    Description: API input payload for /analyze.
    Layer: L1
    Input: resume_text + job payload
    Output: triggers pipeline
    """
    resume_text: str = Field(..., min_length=10)
    job: Dict[str, Any] = Field(..., description="JobDescription-like JSON")
    user_phone: Optional[str] = None


class AnalyzeResponse(BaseModel):
    """
    Description: API response for /analyze.
    Layer: L1
    Input: Pipeline result
    Output: run_id + key artifact paths
    """
    run_id: str
    status: str
    artifacts: Dict[str, Any]


def run_pipeline(*, resume_text: str, job_payload: Dict[str, Any], user_phone: Optional[str] = None) -> OrchestrationState:
    """
    Description: Execute L2–L9 pipeline (local-first) and persist artifacts/state for UI polling.
    Layer: L2-L9
    Input: resume_text + job payload
    Output: OrchestrationState
    """
    settings = get_settings()
    notifier = NotificationService(dry_run=not (settings.twilio_account_sid and settings.user_phone))
    sec = SanitizeAgent()

    st = OrchestrationState.new(env=settings.environment, mode="agentic", git_sha=None)
    st.meta.update({"w1_skill_overlap": 0.45, "w2_experience_alignment": 0.35, "w3_ats_score": 0.20})
    if user_phone:
        st.meta["user_phone_override"] = user_phone

    # L0 Security
    st.start_step("l0_security", layer_id="L0", tool_name="sanitize_before_llm", input_ref={})
    safe = sec.sanitize_before_llm(
        state=st, step_id="l0_security", tool_name="sanitize_before_llm", user_text=resume_text, context="resume"
    )
    if safe is None:
        persist_state(st)
        # Notify user on block (security)
        if (user_phone or settings.user_phone):
            notifier.notify(state=st, event="needs_human_approval", to_phone=(user_phone or settings.user_phone))
        return st
    st.end_step("l0_security", status="ok", output_ref={"sanitized": True}, message="security_pass")

    # Parse JobDescription
    try:
        job = JobDescription(**job_payload)
    except Exception as e:
        st.status = "failed"
        st.meta["run_failure_code"] = "BAD_JOB_PAYLOAD"
        st.meta["run_failure_detail"] = str(e)
        persist_state(st)
        return st

    # Seed evaluator keyword context
    st.meta["target_role_keywords"] = list(set((job.required_skills or []) + (job.preferred_skills or [])))
    st.meta["target_requirements_text"] = job.requirements_text
    if job.market_competition_factor:
        st.meta["market_competition_factor"] = float(job.market_competition_factor)

    out_dir = _run_dir(st.run_id)

    # Save raw inputs as artifacts
    (out_dir / "resume_raw.txt").write_text(safe, encoding="utf-8")
    (out_dir / "job.json").write_text(json.dumps(job.model_dump(), indent=2), encoding="utf-8")
    st.add_artifact("resume_raw", str(out_dir / "resume_raw.txt"), content_type="text/plain")
    st.add_artifact(f"job_{job.job_id}", str(out_dir / "job.json"), content_type="application/json")
    persist_state(st)

    # L2 Parser + L3 Evaluator recursive gate
    parser = ParserAgentService()
    parser_eval = ParserEvaluatorService()
    feedback = []
    extracted = None
    for attempt in range(4):
        sid = f"l2_parse_{attempt+1}"
        st.start_step(sid, layer_id="L2", tool_name="parser_agent_service", input_ref={"attempt": attempt+1})
        extracted = parser.parse(raw_text=safe, orchestration_state=st, feedback=feedback)
        p = out_dir / f"extracted_resume_attempt_{attempt+1}.json"
        p.write_text(json.dumps(extracted.to_json_dict(), indent=2), encoding="utf-8")
        st.add_artifact(f"extracted_resume_attempt_{attempt+1}", str(p), content_type="application/json")
        st.end_step(sid, status="ok", output_ref={"artifact_key": f"extracted_resume_attempt_{attempt+1}"}, message="parsed")

        ev = parser_eval.evaluate(
            orchestration_state=st,
            raw_text=safe,
            extracted=extracted,
            target_id="resume_main",
            threshold=0.80,
            retry_count=attempt,
            max_retries=3,
        )
        decision = st.apply_recursive_gate(target_id="resume_main", layer_id="L3")
        persist_state(st)
        if decision == "pass":
            break
        if decision == "human_approval":
            # Notify user
            st.status = "needs_human_approval"
            persist_state(st)
            if (user_phone or settings.user_phone):
                notifier.notify(state=st, event="needs_human_approval", to_phone=(user_phone or settings.user_phone))
            return st
        feedback = ev.feedback

    # L4 Match
    matcher = MatcherAgentService()
    st.start_step("l4_match", layer_id="L4", tool_name="matcher_agent_service", input_ref={"job_id": job.job_id})
    report = matcher.match(resume=extracted, job=job, orchestration_state=st)
    mr = out_dir / f"match_report_{job.job_id}.json"
    mr.write_text(json.dumps(report.model_dump(), indent=2), encoding="utf-8")
    st.add_artifact(f"match_report_{job.job_id}", str(mr), content_type="application/json")
    st.meta.setdefault("job_scores", {})
    st.meta.setdefault("job_components", {})
    st.meta.setdefault("job_meta", {})
    st.meta["job_scores"][job.job_id] = float(report.interview_chance_score)
    st.meta["job_components"][job.job_id] = report.components.model_dump()
    st.meta["job_meta"][job.job_id] = {"role_title": job.role_title, "company": job.company}
    st.end_step("l4_match", status="ok", output_ref={"artifact_key": f"match_report_{job.job_id}"}, message="matched")
    persist_state(st)

    # L5 Strategy
    strategist = StrategyAgentService()
    st.start_step("l5_strategy", layer_id="L5", tool_name="strategy_agent_service", input_ref={"job_id": job.job_id})
    strategy = strategist.generate(resume=extracted, job=job, match_report=report, orchestration_state=st, feedback=[])
    sp = out_dir / f"pivot_strategy_{job.job_id}.json"
    sp.write_text(json.dumps(strategy.model_dump(), indent=2), encoding="utf-8")
    st.add_artifact(f"pivot_strategy_{job.job_id}", str(sp), content_type="application/json")
    st.end_step("l5_strategy", status="ok", output_ref={"artifact_key": f"pivot_strategy_{job.job_id}"}, message="strategy")
    persist_state(st)

    # L6 Cover Letter
    cover = CoverLetterService()
    st.start_step("l6_cover", layer_id="L6", tool_name="cover_letter_service", input_ref={"job_id": job.job_id})
    draft = cover.draft(
        resume=extracted, job=job, match_report=report, orchestration_state=st, feedback=["Include contact header"]
    )
    cp = out_dir / f"cover_letter_{job.job_id}.md"
    cp.write_text(draft.body, encoding="utf-8")
    st.add_artifact(f"cover_letter_{job.job_id}", str(cp), content_type="text/markdown")
    st.end_step("l6_cover", status="ok", output_ref={"artifact_key": f"cover_letter_{job.job_id}"}, message="cover_letter")
    persist_state(st)

    # L7 Apply (simulated) — only after this do we mark completed (ApplyExecutorService does it)
    apply_exec = ApplyExecutorService()
    st.start_step("l7_apply", layer_id="L7", tool_name="apply_executor_service", input_ref={"job_id": job.job_id})
    submission = apply_exec.submit(
        orchestration_state=st,
        job_id=job.job_id,
        resume_artifact_key="extracted_resume_attempt_1",
        cover_letter_artifact_key=f"cover_letter_{job.job_id}",
        notes="Beta submit (simulated).",
    )
    subp = out_dir / f"submission_{submission.submission_id}.json"
    subp.write_text(json.dumps(submission.model_dump(), indent=2), encoding="utf-8")
    st.add_artifact(f"submission_{submission.submission_id}", str(subp), content_type="application/json")
    st.end_step("l7_apply", status="ok", output_ref={"submission_id": submission.submission_id}, message="submitted")
    persist_state(st)

    # L9 XAI outputs + dossier export
    xai = XAIService()
    xai_paths = xai.write_outputs(state=st, require_reportlab=False)
    st.add_artifact("transparency_matrix_json", xai_paths["json"], content_type="application/json")
    st.add_artifact("xai_transparency_pdf", xai_paths["pdf"], content_type="application/pdf")

    exporter = CareerDossierExporter()
    bundle = exporter.bundle_reports(run_id=st.run_id, final_pdf_path=xai_paths["pdf"])
    st.add_artifact("career_dossier_zip", bundle["zip"], content_type="application/zip")
    persist_state(st)

    # Notify completion
    if (user_phone or settings.user_phone):
        notifier.notify(state=st, event="completed", to_phone=(user_phone or settings.user_phone))

    return st


@app.post("/analyze", response_model=AnalyzeResponse)
def analyze(req: AnalyzeRequest) -> AnalyzeResponse:
    """
    Description: Trigger L2–L9 pipeline for a resume + job payload.
    Layer: L1
    Input: AnalyzeRequest
    Output: AnalyzeResponse (run_id + artifacts)
    """
    st = run_pipeline(resume_text=req.resume_text, job_payload=req.job, user_phone=req.user_phone)
    persist_state(st)
    return AnalyzeResponse(run_id=st.run_id, status=st.status, artifacts={k: v.model_dump() for k, v in st.artifacts.items()})


@app.get("/status/{run_id}")
def status(run_id: str) -> Dict[str, Any]:
    """
    Description: Poll the persisted OrchestrationState for a run_id.
    Layer: L1
    Input: run_id
    Output: State JSON dict
    """
    try:
        return load_state(run_id)
    except FileNotFoundError:
        raise HTTPException(status_code=404, detail="run_id not found")
'''
Path("src/careeragent/api/main.py").write_text(api_main_py.strip() + "\n", encoding="utf-8")

# -----------------------------
# 4) Streamlit Frontend: app/main.py
# -----------------------------
streamlit_py = r'''
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, Optional

import requests
import streamlit as st

API_BASE = st.secrets.get("API_BASE", "http://127.0.0.1:8000")


def _download_button_from_path(label: str, path: str, mime: str) -> None:
    p = Path(path)
    if not p.exists():
        st.warning(f"Missing file: {path}")
        return
    st.download_button(label=label, data=p.read_bytes(), file_name=p.name, mime=mime)


st.set_page_config(page_title="CareerAgent-AI Beta", layout="wide")
st.title("CareerAgent-AI — Beta Dashboard")

with st.sidebar:
    st.subheader("Backend")
    API_BASE = st.text_input("API Base URL", value=API_BASE)
    st.caption("FastAPI Brain must be running.")
    st.divider()
    st.subheader("Inputs")

resume_text = st.text_area("Resume (raw text)", height=220, placeholder="Paste resume text here...")
job_json = st.text_area(
    "Job (JobDescription JSON)",
    height=220,
    value=json.dumps(
        {
            "job_id": "job_001",
            "role_title": "Data Scientist",
            "company": "ExampleCo",
            "country_code": "US",
            "required_skills": ["python", "sql", "fastapi"],
            "preferred_skills": ["mlflow", "docker"],
            "requirements_text": "python sql fastapi mlflow docker production",
            "applicants_count": 200,
        },
        indent=2,
    ),
)
user_phone = st.text_input("Your phone (optional, Twilio)", value="")

colA, colB = st.columns([1, 1])
with colA:
    run_btn = st.button("Run Analyze", type="primary", use_container_width=True)
with colB:
    run_id_input = st.text_input("Run ID (to poll)", value="")

if run_btn:
    try:
        job = json.loads(job_json)
    except Exception as e:
        st.error(f"Invalid Job JSON: {e}")
        st.stop()

    if not resume_text.strip():
        st.error("Paste resume text.")
        st.stop()

    payload = {"resume_text": resume_text, "job": job, "user_phone": (user_phone.strip() or None)}
    r = requests.post(f"{API_BASE}/analyze", json=payload, timeout=120)
    if r.status_code >= 400:
        st.error(f"/analyze failed: {r.status_code} {r.text[:500]}")
        st.stop()

    data = r.json()
    st.session_state["run_id"] = data["run_id"]
    st.success(f"Started run: {data['run_id']} (status: {data['status']})")

run_id = st.session_state.get("run_id") or run_id_input.strip()
if run_id:
    st.subheader(f"Run: {run_id}")

    poll = st.button("Poll Status", use_container_width=True)
    if poll or True:
        r = requests.get(f"{API_BASE}/status/{run_id}", timeout=30)
        if r.status_code == 404:
            st.warning("Run not found yet.")
        else:
            st_data: Dict[str, Any] = r.json()

            # Progress visualization from steps
            steps = st_data.get("steps", [])
            total = max(1, len(steps))
            done = sum(1 for s in steps if s.get("finished_at_utc"))
            st.progress(done / total)

            status = st_data.get("status", "unknown")
            st.metric("Run Status", status)

            # Match score gauge (simple)
            meta = st_data.get("meta", {}) or {}
            job_scores = meta.get("job_scores", {}) or {}
            if job_scores:
                job_id, score = next(iter(job_scores.items()))
                pct = float(score) * 100.0
                st.metric("Match Score (InterviewChance)", f"{pct:.2f}%")
                st.progress(min(1.0, max(0.0, pct / 100.0)))

            # Audit trail
            with st.expander("Audit Trail (Steps)"):
                for s in steps:
                    st.write(f"- [{s.get('layer_id')}] {s.get('tool_name')} | {s.get('status')} | {s.get('step_id')}")

            # Downloads (dossier zip + XAI PDF)
            artifacts = st_data.get("artifacts", {}) or {}
            zip_ref = artifacts.get("career_dossier_zip")
            pdf_ref = artifacts.get("xai_transparency_pdf")

            d1, d2 = st.columns(2)
            with d1:
                if zip_ref and zip_ref.get("path"):
                    _download_button_from_path("Download Career Dossier (ZIP)", zip_ref["path"], "application/zip")
            with d2:
                if pdf_ref and pdf_ref.get("path"):
                    _download_button_from_path("Download XAI Transparency (PDF)", pdf_ref["path"], "application/pdf")

            with st.expander("Full State JSON"):
                st.json(st_data)
'''
Path("app/main.py").write_text(streamlit_py.strip() + "\n", encoding="utf-8")

# -----------------------------
# 5) run_app.py launcher
# -----------------------------
run_app_py = r'''
from __future__ import annotations

import subprocess
import sys
import time


def main() -> int:
    """
    Description: Launch FastAPI (uvicorn) and Streamlit dashboard locally.
    Layer: L0
    Input: None
    Output: process exit code
    """
    # Prefer correct module path: careeragent.api.main:app
    uvicorn_cmd = [sys.executable, "-m", "uvicorn", "careeragent.api.main:app", "--host", "127.0.0.1", "--port", "8000"]
    streamlit_cmd = [sys.executable, "-m", "streamlit", "run", "app/main.py", "--server.port", "8501"]

    print("Starting FastAPI:", " ".join(uvicorn_cmd))
    api = subprocess.Popen(uvicorn_cmd)

    # small delay so backend boots
    time.sleep(1.2)

    print("Starting Streamlit:", " ".join(streamlit_cmd))
    ui = subprocess.Popen(streamlit_cmd)

    try:
        api.wait()
        ui.wait()
        return 0
    except KeyboardInterrupt:
        print("Stopping...")
        api.terminate()
        ui.terminate()
        return 130


if __name__ == "__main__":
    raise SystemExit(main())
'''
Path("run_app.py").write_text(run_app_py.strip() + "\n", encoding="utf-8")

# -----------------------------
# 6) Environment sync: patch pyproject.toml dependencies
# -----------------------------
pyproject = Path("pyproject.toml")
if pyproject.exists():
    raw = pyproject.read_text(encoding="utf-8")

    # naive but safe-ish: ensure required deps appear in [project].dependencies list
    required = [
        "fastapi",
        "uvicorn",
        "streamlit",
        "reportlab",
        "twilio",
        "pydantic-settings",
    ]

    # Find dependencies array block
    dep_block = re.search(r'(?s)\[project\].*?dependencies\s*=\s*\[(.*?)\]\s*', raw)
    if dep_block:
        inside = dep_block.group(1)
        existing = set(re.findall(r'"([^"]+)"', inside))
        # Keep any version pins if present; check by prefix match before adding
        def has_prefix(pkg: str) -> bool:
            return any(e.lower().startswith(pkg.lower()) for e in existing)

        to_add = []
        for pkg in required:
            if not has_prefix(pkg):
                # minimal pins; keep compatible
                if pkg == "uvicorn":
                    to_add.append('"uvicorn>=0.27"')
                elif pkg == "streamlit":
                    to_add.append('"streamlit>=1.33"')
                elif pkg == "reportlab":
                    to_add.append('"reportlab>=4.0"')
                elif pkg == "twilio":
                    to_add.append('"twilio>=9.0"')
                elif pkg == "pydantic-settings":
                    to_add.append('"pydantic-settings>=2.2"')
                else:
                    to_add.append(f'"{pkg}>=0"')

        if to_add:
            # inject before closing bracket of dependencies list
            new_inside = inside.rstrip()
            if new_inside.strip() and not new_inside.strip().endswith(","):
                new_inside = new_inside + ",\n  " + ",\n  ".join(to_add) + "\n"
            else:
                new_inside = new_inside + "\n  " + ",\n  ".join(to_add) + "\n"
            raw = raw.replace(inside, new_inside)
            pyproject.write_text(raw, encoding="utf-8")
            print("✅ Patched pyproject.toml dependencies:", to_add)
        else:
            print("ℹ️ pyproject.toml already includes required deps.")
    else:
        print("⚠️ Could not find [project].dependencies block to patch. Please add deps manually.")
else:
    print("⚠️ pyproject.toml not found. Add deps to your environment manually.")

# -----------------------------
# 7) Smoke imports (no server start)
# -----------------------------
import sys
sys.path.insert(0, str(Path("src").resolve()))

from careeragent.config import get_settings
from careeragent.api.main import app as fastapi_app

s = get_settings()
assert fastapi_app is not None

print("✅ Batch 9 files written:")
print(" - src/careeragent/config.py")
print(" - src/careeragent/api/main.py")
print(" - app/main.py")
print(" - run_app.py")
print("✅ Smoke imports OK.")

ℹ️ pyproject.toml already includes required deps.
✅ Batch 9 files written:
 - src/careeragent/config.py
 - src/careeragent/api/main.py
 - app/main.py
 - run_app.py
✅ Smoke imports OK.
